<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/manutencao_equipamentos3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
from datetime import datetime, timedelta

# Classe Equipamento
class Equipamento:
    def __init__(self, id, nome, ultima_manutencao, intervalo_manutencao):
        self.id = id
        self.nome = nome
        self.ultima_manutencao = datetime.strptime(ultima_manutencao, '%Y-%m-%d')
        self.intervalo_manutencao = int(intervalo_manutencao)
        self.proxima_manutencao = self.ultima_manutencao + timedelta(days=self.intervalo_manutencao)

    def registrar_manutencao(self, data_manutencao):
        self.ultima_manutencao = datetime.strptime(data_manutencao, '%Y-%m-%d')
        self.proxima_manutencao = self.ultima_manutencao + timedelta(days=self.intervalo_manutencao)

    def to_csv_row(self):
        return [self.id, self.nome, self.ultima_manutencao.strftime('%Y-%m-%d'), self.intervalo_manutencao, self.proxima_manutencao.strftime('%Y-%m-%d')]

    @classmethod
    def from_csv_row(cls, row):
        return cls(row[0], row[1], row[2], row[3])

# Classe GerenciadorManutencao
class GerenciadorManutencao:
    FILE_PATH = 'manutencao_equipamentos.csv'

    def __init__(self):
        self.equipamentos = []
        self._load_from_csv()

    def _load_from_csv(self):
        try:
            with open(self.FILE_PATH, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Pular cabeçalho
                self.equipamentos = [Equipamento.from_csv_row(row) for row in reader]
        except FileNotFoundError:
            self._initialize_csv()

    def _initialize_csv(self):
        with open(self.FILE_PATH, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['ID', 'Nome', 'Ultima_Manutencao', 'Intervalo_Manutencao', 'Proxima_Manutencao'])

    def _save_to_csv(self):
        with open(self.FILE_PATH, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['ID', 'Nome', 'Ultima_Manutencao', 'Intervalo_Manutencao', 'Proxima_Manutencao'])
            for equipamento in self.equipamentos:
                writer.writerow(equipamento.to_csv_row())

    def adicionar_equipamento(self, id, nome, ultima_manutencao, intervalo_manutencao):
        novo_equipamento = Equipamento(id, nome, ultima_manutencao, intervalo_manutencao)
        self.equipamentos.append(novo_equipamento)
        self._save_to_csv()

    def registrar_manutencao(self, id, data_manutencao):
        for equipamento in self.equipamentos:
            if equipamento.id == id:
                equipamento.registrar_manutencao(data_manutencao)
                break
        self._save_to_csv()

    def listar_equipamentos(self):
        for equipamento in self.equipamentos:
            print(f'ID: {equipamento.id}, Nome: {equipamento.nome}, Última Manutenção: {equipamento.ultima_manutencao.strftime("%Y-%m-%d")}, Intervalo de Manutenção (dias): {equipamento.intervalo_manutencao}, Próxima Manutenção: {equipamento.proxima_manutencao.strftime("%Y-%m-%d")}')

    def alertar_manutencao_pendente(self):
        hoje = datetime.today()
        for equipamento in self.equipamentos:
            if equipamento.proxima_manutencao <= hoje:
                print(f'ALERTA: Equipamento {equipamento.nome} (ID: {equipamento.id}) precisa de manutenção. Próxima manutenção era em {equipamento.proxima_manutencao.strftime("%Y-%m-%d")}')


In [2]:
# Exemplo de uso
gerenciador = GerenciadorManutencao()

# Adicionar equipamentos (exemplo)
gerenciador.adicionar_equipamento('1', 'Máquina de Corte CNC', '2024-01-01', 120)
gerenciador.adicionar_equipamento('2', 'Prensa Hidráulica', '2024-01-01', 60)

# Listar equipamentos
print("Equipamentos antes da manutenção:")
gerenciador.listar_equipamentos()

# Registrar manutenção (exemplo)
gerenciador.registrar_manutencao('1', '2024-02-01')

# Listar equipamentos após manutenção
print("\nEquipamentos após a manutenção:")
gerenciador.listar_equipamentos()

# Alertar sobre manutenção pendente
print("\nAlertas de manutenção pendente:")
gerenciador.alertar_manutencao_pendente()


Equipamentos antes da manutenção:
ID: 1, Nome: Máquina de Corte CNC, Última Manutenção: 2024-01-01, Intervalo de Manutenção (dias): 120, Próxima Manutenção: 2024-04-30
ID: 2, Nome: Prensa Hidráulica, Última Manutenção: 2024-01-01, Intervalo de Manutenção (dias): 60, Próxima Manutenção: 2024-03-01

Equipamentos após a manutenção:
ID: 1, Nome: Máquina de Corte CNC, Última Manutenção: 2024-02-01, Intervalo de Manutenção (dias): 120, Próxima Manutenção: 2024-05-31
ID: 2, Nome: Prensa Hidráulica, Última Manutenção: 2024-01-01, Intervalo de Manutenção (dias): 60, Próxima Manutenção: 2024-03-01

Alertas de manutenção pendente:
ALERTA: Equipamento Máquina de Corte CNC (ID: 1) precisa de manutenção. Próxima manutenção era em 2024-05-31
ALERTA: Equipamento Prensa Hidráulica (ID: 2) precisa de manutenção. Próxima manutenção era em 2024-03-01
